# Third Exercise (Chapter 7 & 8)
This exercise focuses on optimization and regularization.

In the optimization part, we will
- implement the Adam optimizer
- compare Adam, SGD, SGD with Momentum
- implement and analyze some learning rate schedules

In the regularization part, we will
- implement Dropout
- implement L1/L2 loss
- analyze the effect of the different regularization methods on the parameter distribution.
- think about early stopping and data augmentation

You can **reuse the code you wrote in the last exercise, or you can use the code we provide (below)**. Just copy the relevant parts into the cell below. Please **note that we extended the `Module` class** with a state to determine whether we're training or evaluating and two functions to toggle this state. We therefore also adapted the training loop function.

The first task of this exercise is to implement Adam, you can **skip forward to the exercises by clicking [here](#adam)**.

### Base classes: `Parameter` and `Module`

In [6]:
from typing import Iterable, List, Optional, Tuple, Callable
import pickle
import gzip

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize

In [7]:
class Parameter:
    """A trainable parameter.

    This class not only stores th_solution-Copy1e value of the parameter but also tensors/
    properties associated with it, such as the gradient of the current backward
    pass.
    """

    def __init__(self, data: np.ndarray, name: Optional[str] = None):
        self.data = data
        self.grad = None
        self.name = name
        self.state_dict = dict()  # dict to store additional, optional information


class Module:
    """The base class all network modules must inherit from."""

    def __init__(self):
        # Cache of the input of the forward pass.
        # We need it during the backward pass in most layers,
        #  e.g., to compute the gradient w.r.t to the weights.
        self.input_cache = None
        self.training = True

    def __call__(self, *args) -> np.ndarray:
        """Alias for forward, convenience function."""
        return self.forward(*args)

    def forward(self, *args) -> np.ndarray:
        """Compute the forward pass through the module.

        Args:
           args: The inputs, e.g., the output of the previous layer.
        """
        raise NotImplementedError

    def backward(self, grad: np.ndarray) -> np.ndarray:
        """Compute the backward pass through the module.

        This method computes the gradients with respect to the trainable
        parameters and with respect to the first input.
        If the module has trainable parameters, this method needs to update
        the respective parameter.grad property.

        Args:
            grad: The gradient of the following layer.

        Returns:
            The gradient with respect to the first input argument. In general
            it might be useful to return the gradients w.r.t. to all inputs, we
            omit this here to keep things simple.
        """
        raise NotImplementedError

    def parameters(self) -> List[Parameter]:
        """Return the module parameters."""
        return []  # default to empty list

    def train(self, mode : bool = True) -> 'Module':
        """Set the module to training mode.

        This only affects some Modules, such as Dropout.
        
        Returns:
            self.
        """
        self.training = mode
        return self

    def eval(self) -> 'Module':
        """Set the module to evaluation mode.

        This only affects some Modules, such as Dropout.

        Returns:
            self.
        """
        return self.train(False)

    def check_gradients(self, input_args: Tuple[np.ndarray]):
        """Verify the implementation of the gradients.

        This includes the gradient with respect to the input as well as the
        gradients w.r.t. the parameters if the module contains any.

        As the scipy grad check only works on scalar functions, we compute
        the sum over the output to obtain a scalar.
        """
        assert isinstance(input_args, tuple), (
            "input_args must be a tuple but is {}".format(type(input_args)))
        TOLERANCE = 1e-6
        self.check_gradients_wrt_input(input_args, TOLERANCE)
        self.check_gradients_wrt_params(input_args, TOLERANCE)

    def _zero_grad(self):
        """(Re-) intialize the param's grads to 0. Helper for grad checking."""
        for p in self.parameters():
            p.grad = np.zeros_like(p.data)

    def check_gradients_wrt_input(self, input_args: Tuple[np.ndarray],
                                  tolerance: float):
        """Verify the implementation of the module's gradient w.r.t. input."""

        def output_given_input(x: np.ndarray):
            """Wrap self.forward for scipy.optimize.check_grad."""
            # we only compute the gradient w.r.t. to the first input arg.
            args = (x.reshape(input_args[0].shape),) + input_args[1:]
            return np.sum(self.forward(*args))

        def grad_given_input(x: np.ndarray):
            """Wrap self.backward for scipy.optimize.check_grad."""
            self._zero_grad()
            # run self.forward to store the new input
            args = (x.reshape(input_args[0].shape),) + input_args[1:]
            out = self.forward(*args)
            # compute the gradient w.r.t. to the input
            return np.ravel(self.backward(np.ones_like(out)))

        error = scipy.optimize.check_grad(
            output_given_input, grad_given_input, np.ravel(input_args[0]))
        num_outputs = np.prod(self.forward(*input_args).shape)
        if np.squeeze(error) / num_outputs > tolerance:
            raise RuntimeError("Check of gradient w.r.t. to input for {} failed."
                               "Error {:.4E} > {:.4E}."
                               .format(self, np.squeeze(error), tolerance))

    def check_gradients_wrt_params(self, input_args: Tuple[np.ndarray],
                                   tolerance: float):
        """Verify the implementation of the module's gradient w.r.t. params."""
        for param in self.parameters():
            def output_given_params(new_param: np.ndarray):
                """Wrap self.forward, change the parameters to new_param."""
                param.data = new_param.reshape(param.data.shape)
                return np.sum(self.forward(*input_args))

            def grad_given_params(new_param: np.ndarray):
                self._zero_grad()
                param.data = new_param.reshape(param.data.shape)
                out = self.forward(*input_args)
                # compute the gradient w.r.t. to param
                self.backward(np.ones_like(out))
                return np.ravel(param.grad)
            # flatten the param as scipy can only handle 1D params
            param_init = np.ravel(np.copy(param.data))
            error = scipy.optimize.check_grad(output_given_params,
                                              grad_given_params,
                                              param_init)
            num_outputs = np.prod(self.forward(*input_args).shape)
            if np.squeeze(error) / num_outputs > tolerance:
                raise RuntimeError("Check of gradient w.r.t. to param '{}' for"
                                   "{} failed. Error {:.4E} > {:.4E}."
                                   .format(param.name, self, error, tolerance))



### Activation functions: `Relu` and `Softmax`

In [8]:
class Softmax(Module):

    def forward(self, z: np.ndarray) -> np.ndarray:
        # Shift input for numerical stability.
        reduction_axes = tuple(range(1, len(z.shape)))
        shift_z = z - np.max(z, axis=reduction_axes, keepdims=True)
        exps = np.exp(shift_z)
        h = exps / np.sum(exps, axis=reduction_axes, keepdims=True)
        return h

    def backward(self, grad) -> np.ndarray:
        error_msg = ("Softmax doesn't need to implement a gradient here, as it's"
                     "only needed in CrossEntropyLoss, where we can simplify"
                     "the gradient for the combined expression.")
        raise NotImplementedError(error_msg)


class Relu(Module):

    def forward(self, z: np.ndarray) -> np.ndarray:
        self.input_cache = z
        return np.maximum(0, z)

    def backward(self, grad: np.ndarray) -> np.ndarray:
        z = self.input_cache
        return grad.reshape(z.shape) * np.where(z > 0, 1, 0)

### Layers

In [9]:
class Linear(Module):

    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.W = Parameter(np.random.randn(out_features, in_features) * 0.01,
                           name="W")
        self.b = Parameter(np.ones((out_features, 1)) * 0.01, name="b")

    def forward(self, x: np.ndarray) -> np.ndarray:
        assert len(x.shape) == 3, ("x.shape should be (batch_size, input_size, 1)"
                                   " but is {}.".format(x.shape))
        self.input_cache = x
        z = self.W.data @ x + self.b.data
        return z

    def backward(self, grad: np.ndarray) -> np.ndarray:
        x = self.input_cache
        # remember that we have a batch dimension when transposing, i.e.,
        # we need to use np.transpose instead of array.T
        self.W.grad += np.sum(grad @ np.transpose(x, [0, 2, 1]), axis=0)
        self.b.grad += np.sum(grad, axis=0)
        return self.W.data.T @ grad

    def parameters(self) -> List[Parameter]:
        return self.W, self.b


class Sequential(Module):
    """A sequential container to stack modules.

    Modules will be added to it in the order they are passed to the
    constructor.

    Example network with one hidden layer:
    model = Sequential(
                  Linear(5,10),
                  ReLU(),
                  Linear(10,10),
                )
    """
    def __init__(self, *args: List[Module]):
        super().__init__()
        self.modules = args

    def forward(self, x: np.ndarray) -> np.ndarray:
        for module in self.modules:
            x = module(x)  # equivalent to module.forward(x)
        return x

    def backward(self, grad: np.ndarray) -> np.ndarray:
        for module in reversed(self.modules):
            grad = module.backward(grad)
        return grad

    def parameters(self) -> List[Parameter]:
        # iterate over modules and retrieve their parameters, iterate over
        # parameters to flatten the list
        return [param for module in self.modules
                for param in module.parameters()]
    
    def train(self, mode: bool = True) -> 'Sequential':
        """Set the train mode of the Sequential module and it's sub-modules.
        
        This only affects some modules, e.g., Dropout.
        
        Returns:
            self.
        """
        for module in self.modules:
            module.train(mode)
        return self


def one_hot_encoding(y: np.ndarray, num_classes: int) -> np.ndarray:
    """Convert integer labels to one hot encoding.

    Example: y=[1, 2] --> [[0, 1, 0], [0, 0, 2]]
    """
    encoded = np.zeros(y.shape + (num_classes,))
    encoded[np.arange(len(y)), y] = 1
    return encoded

### Optimizers

In [10]:
class Optimizer:
    """The base class for optimizers.

    All optimizers must implement a step() method that updates the parameters.
    The general optimization loop then looks like this:

    for inputs, targets in dataset:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

    `zero_grad` initializes the gradients of the parameters to zero. This
    allows to accumulate gradients (instead of replacing it) during
    backpropagation, which is e.g. useful for skip connections.
    """

    def __init__(self, params: Iterable[Parameter]):
        """
        Args:
            params: The parameters to be optimized.
        """
        self._params = params

    def step(self) -> None:
        """Update the parameters."""
        raise NotImplementedError

    def zero_grad(self) -> None:
        """Clear the gradients of all optimized parameters."""
        for param in self._params:
            assert isinstance(param, Parameter)
            param.grad = np.zeros_like(param.data)


class SGD(Optimizer):
    """Stochastic Gradient Descent (SGD) optimizer with optional Momentum."""

    def __init__(self, params: Iterable[Parameter], lr: float,
                 momentum: Optional[float] = None):
        super().__init__(params)
        self.lr = lr
        self.momentum = momentum
        if momentum:
            for param in self._params:
                param.state_dict["momentum"] = np.zeros_like(param.data)

    def step(self):
        for p in self._params:
            if self.momentum:
                # update the momentum
                p.state_dict["momentum"] *= self.momentum
                p.state_dict["momentum"] -= self.lr * p.grad
                # update the parameter
                p.data += p.state_dict["momentum"]
            else:
                p.data -= self.lr * p.grad

### Loss Functions


In [11]:
class CrossEntropyLoss(Module):
    """Compute the cross entropy."""

    def __init__(self):
        super().__init__()
        self.softmax = Softmax()

    def forward(self, a: np.ndarray, y: np.ndarray) -> np.ndarray:
        """Compute the cross entropy, mean over batch size."""
        a = self.softmax(a)
        self.input_cache = a, y
        # compute the mean over the batch
        return -np.sum(np.log(a[y == 1])) / len(a)

    def backward(self, _=None) -> np.ndarray:
        # we introduce the argument _ here, to have a unified interface with
        # other Module objects. This simplifies code for gradient checking. 
        # We don't need this arg.
        a, y = self.input_cache
        grad = (a - y) / len(a)

        # We have to recreate the batch dimension
        grad = np.expand_dims(grad, -1)
        return grad

### Training loops

In [56]:
def accuracy(y, predictions, y_is_onehot: bool = False) -> float:
    y_predicted = np.argmax(predictions, axis=-1)
    y = np.argmax(y, axis=-1)
    return np.sum(np.equal(y_predicted, y)) / len(y)


def evaluate(data, labels, model, loss_fn, batch_size):
    predictions = []
    eval_cost = 0.
    data_batched = minibatched(data, batch_size)
    labels_batched = minibatched(labels, batch_size)

    for x, y in zip(data_batched, labels_batched):
        # note that when using cross entropy loss, the softmax is included in the
        # loss and we'd need to apply it manually here to obtain the output as probabilities.
        # However, softmax only rescales the outputs and doesn't change the argmax,
        # so we'll skip this here, as we're only interested in the class prediction.
        h_1 = np.squeeze(model(x))
        predictions.append(h_1)
        eval_cost += loss_fn(h_1, y)
    predictions = np.array(predictions).reshape(-1, 10)
    eval_accuracy = accuracy(y_val, predictions, False)
    return eval_accuracy, eval_cost


def train(model, loss_fn, optimizer, x_train, y_train, x_val, y_val,
          num_epochs, batch_size, scheduler=None):
    train_costs, train_accuracies = np.zeros(num_epochs), np.zeros(num_epochs)
    eval_costs, eval_accuracies = np.zeros(num_epochs), np.zeros(num_epochs)
    ix = np.arange(len(x_train))
    
    for epoch in range(num_epochs):
        print("Epoch {} / {}:".format(epoch + 1, num_epochs))
        print("LR---> ",optimizer.lr)
        training_predictions = []
        
        np.random.shuffle(ix)
        x_train_batched = minibatched(x_train[ix], batch_size)
        y_train_batched = minibatched(y_train[ix], batch_size)
        
        # train for one epoch
        model.train()
        for x_batch, y_batch in zip(x_train_batched, y_train_batched):
            optimizer.zero_grad()
            y_batch_predicted = model(x_batch)
            h_1 = np.squeeze(y_batch_predicted)
            training_predictions.append(h_1)
            loss = loss_fn(h_1, y_batch)
            grad = loss_fn.backward()
            model.backward(grad)
            optimizer.step()
            train_costs[epoch] += loss
        if scheduler:
            scheduler.step()
        model.eval()
       
        training_predictions = np.array(training_predictions).reshape(-1, 10)
        train_accuracies[epoch] = accuracy(y_train[ix], training_predictions, False)
        print("  Training Accuracy: {:.4f}".format(train_accuracies[epoch]))
        print("  Training Cost: {:.4f}".format(train_costs[epoch]))
        eval_accuracies[epoch], eval_costs[epoch] = evaluate(x_val, y_val, model, loss_fn, batch_size)
        print("  Eval Accuracy: {:.4f}".format(eval_accuracies[epoch]))
    return train_costs, train_accuracies, eval_costs, eval_accuracies

### Data Loading

In [25]:
def load_mnist_data():
    """Loads the data, returns training_data, validation_data, test_data."""
    with gzip.open('mnist.pkl.gz', 'rb') as f:
        return pickle.load(f, encoding='latin1')


def minibatched(data: np.ndarray, batch_size: int) -> List[np.ndarray]:
    assert len(data) % batch_size == 0, ("Data length {} is not multiple of batch size {}"
                                         .format(len(data), batch_size))
    return data.reshape(-1, batch_size, *data.shape[1:])


(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_mnist_data()
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

num_classes = 10
y_train = one_hot_encoding(y_train, num_classes)
y_val = one_hot_encoding(y_val, num_classes)
y_test = one_hot_encoding(y_test, num_classes)

## Optimization

<a id='adam'></a>
### Adam
**Implement the step function** of the adam optimizer.

In [57]:
class Adam(Optimizer):

    def __init__(self, params, lr=0.001, betas=(0.9, 0.999), eps=1e-08):
        super().__init__(params)
        # we stick to the pytorch API, the variable names corresponding
        # to the DL book are given in the comments
        self.lr = lr  # lr is called epsilon in the DL book
        self.betas = betas  # betas are called rho in the DL book
        self.eps = eps  # eps is called delta in the DL book
        self.t = 0
        for param in self._params:
            # first order moment variables, called m in the paper
            param.state_dict["s"] = np.zeros_like(param.data)
            # second order moment variables, called v in the paper
            param.state_dict["r"] = np.zeros_like(param.data)

    def step(self) -> None:
        """Update the parameters and decaying averages of past gradients."""
        # START TODO ################
        self.t += 1
        for param in self._params:
            # first order moment variables, called m in the paper
            s = param.state_dict["s"]
            param.state_dict["s"] = self.betas[0]*s + (1-self.betas[0]) * param.grad
            r = param.state_dict["r"]
            param.state_dict["r"] = self.betas[1]*r + (1-self.betas[1]) * param.grad * param.grad
            # second order moment variables, called v in the paper
            s_hat = s/(1-self.betas[0]**self.t)
            r_hat = r/(1-self.betas[1]**self.t)
            delta_theta = -(self.lr*s_hat)/ (self.eps + np.sqrt(r_hat))
            param.data += delta_theta
        # END TODO###################


### Compare: Adam vs SGD vs SGD with Momentum

**Train three models** (30 hidden units, relu) for 10 epochs, one with *Adam*, one with *SGD* and one with *SGD with momentum*. 

In [23]:
linear_units = 30
batch_size = 50
num_epochs = 10
sgd_learning_rate = 0.1
momentum = 0.9
adam_learning_rate = 0.01

# START TODO ################

print("SGD without momentum------------->")
model = Sequential(Linear(784,linear_units), Relu(), Linear(linear_units,10))
params = model.parameters()
optim = SGD(params, sgd_learning_rate)
loss_fn = CrossEntropyLoss()

sgd_cst,sgd_acc,sgdVal_cst,sgdVal_acc=train(model, loss_fn, optim, x_train, y_train, x_val, y_val, num_epochs, batch_size);

print("SGD with momentum------------->")
model = Sequential(Linear(784,linear_units), Relu(), Linear(linear_units,10))
params = model.parameters()
optim = SGD(params, sgd_learning_rate, momentum)
loss_fn = CrossEntropyLoss()

sgdm_cst,sgdm_acc,sgdmVal_cst,sgdmVal_acc=train(model, loss_fn, optim, x_train, y_train, x_val, y_val, num_epochs, batch_size);

print("Optimization with Adam------------->")
model = Sequential(Linear(784,linear_units), Relu(), Linear(linear_units,10))
params = model.parameters()
optim = Adam(params, adam_learning_rate)
loss_fn = CrossEntropyLoss()

ad_cst,ad_acc,adVal_cst,adVal_acc=train(model, loss_fn, optim, x_train, y_train, x_val, y_val, num_epochs, batch_size);
# END TODO###################

SGD without momentum------------->
Epoch 1 / 10:
  Training Accuracy: 0.8285
  Training Cost: 585.8209
  Eval Accuracy: 0.9211
Epoch 2 / 10:
  Training Accuracy: 0.9185
  Training Cost: 277.5401
  Eval Accuracy: 0.9348
Epoch 3 / 10:
  Training Accuracy: 0.9355
  Training Cost: 221.6852
  Eval Accuracy: 0.9445
Epoch 4 / 10:
  Training Accuracy: 0.9458
  Training Cost: 183.6589
  Eval Accuracy: 0.9546
Epoch 5 / 10:
  Training Accuracy: 0.9529
  Training Cost: 157.6582
  Eval Accuracy: 0.9574
Epoch 6 / 10:
  Training Accuracy: 0.9582
  Training Cost: 138.7431
  Eval Accuracy: 0.9627
Epoch 7 / 10:
  Training Accuracy: 0.9619
  Training Cost: 124.6281
  Eval Accuracy: 0.9627
Epoch 8 / 10:
  Training Accuracy: 0.9661
  Training Cost: 113.0037
  Eval Accuracy: 0.9660
Epoch 9 / 10:
  Training Accuracy: 0.9688
  Training Cost: 104.3147
  Eval Accuracy: 0.9663
Epoch 10 / 10:
  Training Accuracy: 0.9715
  Training Cost: 96.0000
  Eval Accuracy: 0.9654
SGD with momentum------------->
Epoch 1 / 10:

**Create two plots**, one for the training loss curves and one for the training accuracies curves.

In [ ]:
import matplotlib.pyplot as plt 
% matplotlib inline

# START TODO ################

# END TODO###################D

### Learning Rate Schedules

Decreasing the learning rate can improve performance of the model.

Pytorch implements learning rate schedules as wrappers for the optimizer and requires the user to call scheduler.step() after each epoch. (We've already done this for you in the training loop above)

**Implement the learning rate scheduler** `PieceWiseConstantLR` and `CosineAnnealingLR`. You can use the provided `LambdaLR` class for this, which works analogously to the pytorch [LambdaLR](https://pytorch.org/docs/master/optim.html#torch.optim.lr_scheduler.LambdaLR)


In [67]:
class LambdaLR:

    def __init__(self, optimizer: Optimizer, lr_lambda: Callable[[int], float]):
        """Sets the learning rate to the initial lr times a given function.

        Args:
            optimizer: The optimzier to wrap.
            lr_lambda: A function that takes the current epoch as an argument
                and returns the corresponding learning rate.
        """
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer
        self.last_epoch = 0
        self.initial_lr = np.copy(optimizer.lr)
        self.lr_lambda = lr_lambda
        print("LR initial======", self.initial_lr)

    def step(self):
        """To be called after each epoch. Update optimizer.lr"""
        self.last_epoch += 1
        self.optimizer.lr = self.lr_lambda(self.last_epoch)
        print("SCHEDULED LR-------",self.optimizer.lr)


class PiecewiseConstantLR(LambdaLR):

    def __init__(self, optimizer: Optimizer, epochs: List[int],
                 learning_rates: List[float]):
        """Set learning rate as piecewise constant steps.

        This class inherits from LambdaLR and implements the lambda
        function that maps the current epoch to the learning rate
        according to epochs.
        
        optimizer: The optimizer to wrap
        """
        def lr_lambda(num_epo):
            i = 0
            for epo in epochs:
                if num_epo <= epo:
                    return learning_rates[i] 
                i+=1
        
        super().__init__(optimizer,lr_lambda)
        # START TODO ################
        #raise NotImplementedError
    

        # End TODO ################
        
        
class CosineAnnealingLR(LambdaLR):

    def __init__(self, optimizer: Optimizer, T_max: int):
        """Set learning rate as a cosine decay.

        This class inherits from LambdaLR and implements the lambda
        function that maps the current epoch to the learning rate
        according to epochs.
        
        optimizer: The optimizer to wrap
        T_max:  Maximum number of epochs.
        """
        # START TODO ################
        raise NotImplementedError
        # End TODO ################

To verify your implementation, **plot the learning rate schedules**, with the number of epochs on the x-axis and the learning rate on the y-axis.

In [68]:
num_epochs = 80

linear_units = 30
batch_size = 50
sgd_learning_rate = 0.1
momentum = 0.9
adam_learning_rate = 0.5


model = Sequential(Linear(784,linear_units), Relu(), Linear(linear_units,10))
params = model.parameters()
optim = Adam(params, adam_learning_rate)
picewise_scheduler = PiecewiseConstantLR(optimizer, [10, 20, 40, 50], [0.1, 0.05, 0.01, 0.001])
loss_fn = CrossEntropyLoss()

ad_cst,ad_acc,adVal_cst,adVal_acc=train(model, loss_fn, optim, x_train, y_train, x_val, y_val, num_epochs, batch_size, picewise_scheduler);


#optimizer = Adam([])
#cosine_scheduler = CosineAnnealingLR(optimizer, num_epochs)

# START TODO ################
# plot piecewise lr

# plot cosine lr
# End TODO ################

LR initial====== 0.01
Epoch 1 / 80:
LR--->  0.5


C:\Users\hp\Anaconda3\envs\tensorflow-sessions\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


SCHEDULED LR------- 0.1
  Training Accuracy: 0.1017
  Training Cost: inf
  Eval Accuracy: 0.0967
Epoch 2 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1015
  Training Cost: 2427.1525
  Eval Accuracy: 0.1089
Epoch 3 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1029
  Training Cost: 2436.2019
  Eval Accuracy: 0.0991
Epoch 4 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1042
  Training Cost: 2442.8857
  Eval Accuracy: 0.1089
Epoch 5 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1029
  Training Cost: 2435.9718
  Eval Accuracy: 0.0991
Epoch 6 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1002
  Training Cost: 2456.9054
  Eval Accuracy: 0.1064
Epoch 7 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1025
  Training Cost: 2410.1017
  Eval Accuracy: 0.0967
Epoch 8 / 80:
LR--->  0.5
SCHEDULED LR------- 0.1
  Training Accuracy: 0.1009
  Training Cost: 2445.8027
  Eval Accuracy: 0.1064


KeyboardInterrupt: 

### Your Feedback on Exercise 3.1

TODO

## Regularization

In this part of the exercise we will take a look multiple regularizers.

### Dropout
**Fill in the missing gaps** for the implementation of the dropout regularization (Chapter 7.12 in the DL book).
During training, the dropout layer randomly sets the input tensor to 0 with probability p and scales the remaining values accordingly. During evaluation, the dropout layer returns the identity.


In [ ]:
class Dropout(Module):
    """Set input elements to zero during training with probability p."""

    def __init__(self, p : float = 0.5, fix_seed=False):
        """
        Args:
            p: Probability of an element to be zeroed. Default: 0.5.
            fix_seed: If true, we always use the same seed in the forward pass.
                This is only needed for gradient checking and should only be
                set True for gradient checking.
        """
        super().__init__()
        self.p = p
        # START TODO ################
        # self.scale = TODO
        # END TODO ################
        self.fix_seed = fix_seed

    def forward(self, x: np.ndarray) -> np.ndarray:
        """Apply dropout during training.
        
        Set values to zero with probability p during training
        and scale them by 1/(1-p). Returns identidy during
        evaluation mode (--> self.training = False).

        Note: This layer should work with all kinds of input shapes.
        """ 
        if not self.training:
            return x
        if self.fix_seed:  # we need this for gradient checking 
            np.random.seed(0)
        # START TODO ################

        # END TODO ################

    def backward(self, grad: np.ndarray) -> np.ndarray:
        if not self.training:
            raise ValueError("Model is set to evaluation mode.")
        # START TODO ################

        # END TODO ################


# Check the gradient implementation
x = np.random.rand(1, 1, 4, 4)
Dropout(fix_seed=True).check_gradients((x,))


### L1/L2 regularization
**Implement $L_1$/$L_2$ regularization**. This is one of the rare cases where we differ from the pytorch API. The reason is that pytorch implements $L_2$ regularization in the optimizer but calles it `weight_decay` (which is actually a different operation if you're not using SGD).


In [ ]:
class L1Regularization(Module):

    def __init__(self, alpha: float, parameters: List[Parameter]):
        super().__init__()
        self.alpha = alpha
        self.params = parameters
    
    def forward(self) -> np.ndarray:
        # START TODO ################

        # END TODO ################

    def backward(self, _=None) -> np.ndarray:
        # START TODO ################

        # END TODO ################
        
    def parameters(self) -> List[Parameter]:
        return self.params

        
class L2Regularization(Module):

    def __init__(self, alpha: float, parameters: List[Parameter]):
        super().__init__()
        self.alpha = alpha
        self.params = parameters
    
    def forward(self) -> np.ndarray:
        # START TODO ################

        # END TODO ################

    def backward(self, _=None) -> np.ndarray:
        # START TODO ################

        # END TODO ################
        
    def parameters(self) -> List[Parameter]:
        return self.params


# Check the gradient implementation. Here we can only check w.r.t. the parameters.
params = [Parameter(np.random.rand(50, 1) * 0.1)]
l2 = L2Regularization(0.1, params)
l1 = L1Regularization(0.1, params)
l2.check_gradients_wrt_params((), 1e-6)
l1.check_gradients_wrt_params((), 1e-6)


class RegularizedCrossEntropy(Module):
    """Combines Cross Entropy loss and Regularization loss by summing them."""
    
    def __init__(self, regularization_loss: Module):
        self.reg_loss = regularization_loss
        self.cross_entropy = CrossEntropyLoss()
        
    def forward(self, a: np.ndarray, y: np.ndarray) -> np.ndarray:
        return self.cross_entropy(a, y) + self.reg_loss()
        
    def backward(self, _=None) -> np.ndarray:
        self.reg_loss.backward()  # this updates parameter gradients, no grad w.r.t input
        return self.cross_entropy.backward()

        

Let's now visualize the effect of regularization on the model parameters.
We'll do that with **four trained models**:
1. Without regularization. It's always a good idea to train your model without any regularization first. Not only to have a baseline but also to check if your model is able to overfit the training data. If it can't overfit, it's likely not powerful enough or there's an error in the implementation. 
2. With L2 regularization, alpha = 0.0001, on the weight parameters (not on the bias)
3. With L1 regularization, alpha = 0.0001, on the weight parameters (not on the bias)
4. With Dropout, drop_probability = 0.3 for the input, drop_probability = 0.5 for the hidden layers.

*Note* that in this case L1/L2/Dropout lead to worse accuracies. This is due to the fact that we use only 30 linear units to keep training times short, which already imposes strong regularization. In practice, we would select a more powerful model. However, the small model still nicely exhibits the effect of regularization on the distribution of the parameters.

In [ ]:
num_epochs = 5
batch_size = 50
learning_rate = 0.05
momentum = 0.9
linear_units = 30
alpha_l1 = 0.0001
alpha_l2 = 0.0001

def build_model():
    return Sequential(Linear(784, linear_units),
                      Relu(),
                      Linear(linear_units, 10))


def build_model_dropout():
    return Sequential(Dropout(0.3),
                      Linear(784, linear_units),
                      Relu(),
                      Dropout(0.5),
                      Linear(linear_units, 10))

models = {}  # dict to store the trained models
# let's save a model, which we won't train, to get the initial parameter distribution
models["before_training"] = build_model()

# no regularization
print("No regularization.")
models["no_reg"] = build_model()
params = [p for p in models["no_reg"].parameters() if "W" in p.name]
cross_entropy = CrossEntropyLoss()
optimizer = SGD(models["no_reg"].parameters(), lr=learning_rate, momentum=momentum)
train(models["no_reg"], cross_entropy, optimizer, x_train, y_train,
      x_val, y_val, num_epochs=num_epochs, batch_size=batch_size)

# START TODO ################


# L2 regularization
print("L2 regularization.")



# L1 regularization
print("L1 regularization.")



# dropout
print("Dropout.")


# END TODO ################
pass

Let's now compare the parameter distribution of the four models by **plotting the histogram of their weight values from -1 to 1 with 100 bins**. 

In [ ]:
# START TODO ################

# End TODO ################

## Data Augmentation

Data augmentation increases model generalization by increasing the training set with fake data.

**Question:** State *five effective operations* to generate fake data on the *MNIST dataset*. State *one operation* which doesn't make sense and why.

**Answer:** 


## Early Stopping

Another very popular technique in deep learning is *early stopping* (deeplearning book, section 7.8). 

![Figure 7.3 from the DeepLearningBook](learning-curve-dl-fig-7-3.png "TEST")

**Questions:**
How do the given loss curves (deep learning book, figure 7.3) relate to *early stopping*?
Why is it a regularization technique? 

**Answers:**


### Your Feedback on Exercise 3.2

TODO